In [16]:
from sys import path
path.append("../")

In [17]:
import jax
import jax.numpy as jnp
from flax.struct import dataclass 

from abc import abstractmethod

In [18]:
@dataclass
class Gaussian:
    info: jnp.ndarray
    precision: jnp.ndarray
    dims: jnp.ndarray 

    @property
    def shape(self):
        return {
            "info": self.info.shape,
            "precision": self.precision.shape,
            "dims": self.dims.shape
        }
 
    @property
    def mean(self) -> jnp.ndarray:
        return jnp.linalg.inv(self.precision) @ self.info
    
    @property
    def covariance(self) -> jnp.ndarray:
        return jnp.linalg.inv(self.precision)
    
    @staticmethod
    def identity(variable: int) -> jnp.ndarray:
        dims = jnp.array([variable, variable, variable, variable])
        return Gaussian(jnp.zeros(4), jnp.eye(4), dims)
    
    def __call__(self, x: jnp.ndarray) -> float:
        return 1/2 * x.T @ self.precision @ x - self.info.T @ x

    def __getitem__(self, index) -> "Gaussian":
        return Gaussian(self.info[index], self.precision[index], self.dims[index])

    def __mul__(self, other: 'Gaussian') -> 'Gaussian':
        if other is None:
            return self.copy()
        
        dims = self.dims.copy()
        other_dims = other.dims.copy()
        other_unique_val = jnp.unique(other_dims, size=other_dims.shape[0]//4)[0]
        
        if dims.shape[0] == other_dims.shape[0] and (dims == other_dims).sum() == dims.shape[0]:
            idxs_self = idxs_other = jnp.arange(len(dims))
        elif dims.shape[0] == 8:
            idxs_self = jnp.arange(len(dims), dtype=int)
            idxs_other = jnp.where(dims == other_unique_val, size=4)[0]
        else:
            idxs_self = jnp.arange(len(dims), dtype=int)
            idxs_other = jnp.arange(len(dims), len(dims) + len(other_dims))
            dims = jnp.concatenate((dims, other_dims))
        
        # Extend self matrix
        prec_self = jnp.zeros((len(dims), len(dims)))
        info_self = jnp.zeros((len(dims), 1))
        
        prec_self = prec_self.at[jnp.ix_(idxs_self, idxs_self)].set(self.precision)
        info_self = info_self.at[jnp.ix_(idxs_self, jnp.zeros(1, dtype=int))].set(self.info.reshape(len(self.dims), 1))
        # Extend other matrix
        prec_other = jnp.zeros((len(dims), len(dims)))
        info_other = jnp.zeros((len(dims), 1))
        
        prec_other = prec_other.at[jnp.ix_(idxs_other, idxs_other)].set(other.precision)
        info_other = info_other.at[jnp.ix_(idxs_other, jnp.zeros((1), dtype=int))].set(other.info.reshape(len(other.dims), 1))
        # Add
        prec = prec_other + prec_self
        info = info_other + info_self
        return Gaussian(info, prec, dims.astype(float))
    
    def marginalize(self, dims_to_remove) -> "Gaussian":
        info, prec = self.info.reshape(-1, 1), self.precision
        
        axis_a = jnp.where(self.dims != dims_to_remove[0], size=4)[0]
        axis_b = jax.lax.select(axis_a[0] == 0, jnp.arange(4, 8), jnp.arange(4))

        info_a = info[jnp.ix_(axis_a, jnp.zeros((1,), dtype=int))]
        prec_aa = prec[jnp.ix_(axis_a, axis_a)]
        info_b = info[jnp.ix_(axis_b, jnp.zeros((1, ), dtype=int))]
        prec_ab = prec[jnp.ix_(axis_a, axis_b)]
        prec_ba = prec[jnp.ix_(axis_b, axis_a)]
        prec_bb = prec[jnp.ix_(axis_b, axis_b)]

        prec_bb_inv = jnp.linalg.inv(prec_bb)
        info_ = info_a - prec_ab @ prec_bb_inv @ info_b
        prec_ = prec_aa - prec_ab @ prec_bb_inv @ prec_ba

        new_dims = jnp.full(4, self.dims[axis_a[0]])
        return Gaussian(info_.flatten(), prec_, new_dims)

# Inter Robot

In [4]:
x = jnp.array([
    [1,1],
    [2,2],
    [3,3],
    [4,4]
])
states = jnp.stack((x, x + 0.1, x + 0.3))
vels = jnp.zeros_like(states)
states = jnp.concat((states, vels), axis=-1)
states, states.shape

(Array([[[1. , 1. , 0. , 0. ],
         [2. , 2. , 0. , 0. ],
         [3. , 3. , 0. , 0. ],
         [4. , 4. , 0. , 0. ]],
 
        [[1.1, 1.1, 0. , 0. ],
         [2.1, 2.1, 0. , 0. ],
         [3.1, 3.1, 0. , 0. ],
         [4.1, 4.1, 0. , 0. ]],
 
        [[1.3, 1.3, 0. , 0. ],
         [2.3, 2.3, 0. , 0. ],
         [3.3, 3.3, 0. , 0. ],
         [4.3, 4.3, 0. , 0. ]]], dtype=float32, weak_type=True),
 (3, 4, 4))

In [31]:
def find_closest_robot(states: jnp.ndarray):
    def find_closest_robot_across_horizon(robot, other_robots):
        closest_index = jnp.argmin(jnp.linalg.norm(robot[0:2] - other_robots[:,0:2], axis=1))
        return closest_index

    def find_batched_closest_robot(batch_states, i):
        modified_states = batch_states.at[i].set(jnp.inf)
        return jax.vmap(find_closest_robot_across_horizon, in_axes=(0, 1))(batch_states[i], modified_states)
    return jax.vmap(find_batched_closest_robot, in_axes=(None, 0))(states, jnp.arange(states.shape[0]))
closest_robots = find_closest_robot(states) # ideally, should be # (N - 1, 4, 2)
closest_robots

Array([[1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1]], dtype=int32)

In [36]:
N_STATES = 4
POSE_NOISE = 1e-15
DYNAMICS_NOISE = 0.005
OBSTACLE_NOISE = 0.005
INTER_ROBOT_NOISE = 0.005

class Factor:
    def __init__(
        self, state: jnp.array, state_precision: jnp.ndarray, dims: jnp.array, linear: bool = True
    ) -> None:
        self._state = state
        self._state_precision = state_precision
        self._linear = linear
        self._dims = dims

    def calculate_likelihood(self) -> Gaussian:
        return Gaussian(
            self._calc_info(self._state, self._state_precision),
            self._calc_precision(self._state, self._state_precision),
            self._dims
        )

    @abstractmethod
    def _calc_measurement(self, state: jnp.ndarray) -> jnp.ndarray:
        pass

    def _calc_info(self, state: jnp.ndarray, precision: jnp.ndarray) -> jnp.ndarray:
        X = state
        if self._linear:
            eta = precision @ (jnp.zeros(N_STATES) - self._calc_measurement(state))
        else:
            J = jax.jacfwd(self._calc_measurement)(state)
            eta = (J.T @ precision) @ (
                (J @ X.reshape((-1,1))) + 0 - self._calc_measurement(state).reshape((-1,1))
            )
        return eta.squeeze()

    def _calc_precision(self, state: jnp.ndarray, precision: jnp.ndarray) -> jnp.ndarray:
        if self._linear:
            return precision
        else:
            J = jax.jacfwd(self._calc_measurement)(state)
            return J.T @ precision @ J

class InterRobotFactor:
    def __init__(
        self,
        state: jnp.ndarray,
        agent_radius: float,
        critical_distance: float,
        t: jnp.ndarray, #ndarray just to hold time,
        dims: jnp.ndarray,
    ) -> None:
        self._critical_distance = critical_distance
        self._agent_radius = agent_radius
        self._z_precision = 100

        dist = self._calc_dist(state[0:4], state[4:])
        dx, dy = (state[0] - state[4])/dist, (state[1] - state[5])/dist
        self._J = jnp.array([[-dx/self._critical_distance, -dy/self._critical_distance, 0, 0,
                              dx/self._critical_distance, dy/self._critical_distance, 0, 0]])

        self._state = state
        # self._precision = jnp.pow(t * INTER_ROBOT_NOISE, -2) * jnp.eye(N_STATES)
        self._state_precision = self._z_precision * jnp.eye(1)
        self._dims = dims
    
    def calculate_likelihood(self) -> Gaussian:
        return Gaussian(
            self._calc_info(self._state, self._state_precision),
            self._calc_precision(self._state_precision),
            self._dims
        )
 
    def _calc_dist(self, state: jnp.array, other_state: jnp.array):
        return jnp.linalg.norm(state[0:2] - other_state[0:2]) 
    
    def _calc_info(self, state: jnp.ndarray, state_precision: jnp.ndarray) -> jnp.ndarray:
        # J = jax.jacfwd(self._calc_measurement)(state).reshape((1,-1))
        # return precision * J * (J @ state + 0 - self._calc_measurement(state))
        return self._J.T @ state_precision @ (self._J @ state[:,jnp.newaxis] - self._calc_measurement(state))
        
    def _calc_precision(self, state_precision: jnp.ndarray) -> jnp.ndarray:
        precision = self._J.T @ state_precision @ self._J 
        # jax.debug.print("before prec diagonal: {}", jnp.diag(prec))
        precision = precision.at[2:4,2:4].set(jnp.eye(2)).at[6:,6:].set(jnp.eye(2))

        # prec_diag_values = jnp.diag(precision)
        # prec_diag_values = jnp.where(prec_diag_values == 0.0, 1.0, prec_diag_values)
        # precision = jnp.fill_diagonal(precision, prec_diag_values, inplace=False)

        jax.debug.print("prec diag: {}", jnp.diag(precision))
        jax.debug.print("inv prec: {}", jnp.linalg.inv(precision))
        jax.debug.print("prec det: {}", jnp.linalg.det(precision))
        jax.debug.breakpoint()
        return precision

    def _calc_measurement(self, state: jnp.ndarray):
        current_state = state[0:4]
        other_state = state[4:]
        dist = self._calc_dist(current_state, other_state)
        measurement = jax.lax.select(
            dist < self._critical_distance, 1.0 - dist / self._critical_distance, 0.
        )
        return measurement

Gaussian(info=Array([nan, nan, nan, nan], dtype=float32), precision=Array([[nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan],
       [nan, nan, nan, nan]], dtype=float32), dims=Array([100., 100., 100., 100.], dtype=float32))

In [ ]:
ir_factor =  InterRobotFactor(jnp.array([-0.1, 0., 0.0, 0.0, 0.1, 0.0, 0.0, 0.0]), 0.2, 1.5, jnp.array([1.]), jnp.array([1., 1., 1., 1., 100., 100., 100., 100.]))
g = ir_factor.calculate_likelihood()
g = g.marginalize(jnp.array([1., 1., 1., 1.]))
g

# Obstacle

In [9]:
obstacle = jnp.array([
    [4.2, 4.2],
    [1.0, 1.0]
])

def find_closest_obstacle(states, obstacles):
    def find_closest_obstacle_for_pt(state_t, obstacles):
        closest_obstacle_idx = jnp.argmin(jnp.linalg.norm(state_t - obstacles, axis=1))
        return obstacles[closest_obstacle_idx]
    def batch_find_closest_obstacle(agent_states, obstacles):
        return jax.vmap(find_closest_obstacle_for_pt, in_axes=(0, None))(agent_states, obstacles)
    return jax.vmap(batch_find_closest_obstacle, in_axes=(0, None))(states, obstacles)
closest_obstacle = find_closest_obstacle(states, obstacle)
closest_obstacle, closest_obstacle.shape

(Array([[[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]],
 
        [[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]],
 
        [[1. , 1. ],
         [1. , 1. ],
         [4.2, 4.2],
         [4.2, 4.2]]], dtype=float32),
 (3, 4, 2))

# Random

In [40]:
x = jnp.array([[1,2,3,4,5,6], [7,8,9,10,11,12]])
y = x * 2
x + y

Array([[ 3,  6,  9, 12, 15, 18],
       [21, 24, 27, 30, 33, 36]], dtype=int32)

In [42]:
x = jnp.arange(32).reshape((8,4))
x[1:-1:2] + x[2:-1:2]

Array([[12, 14, 16, 18],
       [28, 30, 32, 34],
       [44, 46, 48, 50]], dtype=int32)

In [43]:
x[1:-1:2], x[2:-1:2]

(Array([[ 4,  5,  6,  7],
        [12, 13, 14, 15],
        [20, 21, 22, 23]], dtype=int32),
 Array([[ 8,  9, 10, 11],
        [16, 17, 18, 19],
        [24, 25, 26, 27]], dtype=int32))